# Basic liquid handling

In this notebook, you will learn how to use PyHamilton to move water from one range of wells to another.

**Note: before running this notebook, you should have**:

- Installed PyHamilton and the USB driver as described in [the installation guide](installation).
- Connected the Hamilton to your computer using the USB cable.

Video of what this code does:

<iframe width="640" height="360" src="https://www.youtube.com/embed/NN6ltrRj3bU" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

## Setting up a connection with the robot

Start by importing the {class}`~pylabrobot.liquid_handling.liquid_handler.LiquidHandler` class, which will serve as a front end for all liquid handling operations.

Backends serve as communicators between between `LiquidHandler`s and the actual hardware. Since we are using a Hamilton STAR, we also import the {class}`~pylabrobot.liquid_handling.backends.STAR` backend.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pylabrobot.liquid_handling import LiquidHandler
# from pylabrobot.liquid_handling.backends import STAR
from pylabrobot.liquid_handling.backends import OpentronsBackend

# from pylabrobot.liquid_handling.backends.opentrons_backend import OpentronsBackend

In addition, import the {class}`~pylabrobot.resources.hamilton.STARLetDeck`, which represents the deck of the Hamilton STAR.

In [3]:
# from pylabrobot.resources.hamilton import STARLetDeck
from pylabrobot.resources.opentrons.deck import OTDeck

In [4]:
## OPS:: EXTRA PART ADD SEARCH FOR THE CORRECT TYPE OF USB CABLE BY VIEWING IP ADRESSES

Create a new liquid handler using `STAR` as its backend.

In [5]:
# backend = STAR()
# lh = LiquidHandler(backend=backend, deck=STARLetDeck())
# lh = LiquidHandler(backend=OpentronsBackend(host="10.199.253.164"), deck=OTDeck())
lh = LiquidHandler(backend=OpentronsBackend(host="10.199.253.164"), deck=OTDeck(no_trash=True)) # This seems to work
# web: 10.199.253.164 (OT032)
# web: 10.199.253.141 (OT006)
# web: 10.199.253.233 (OT028)
# lan: 169.254.131.46
# backend = OpentronsBackend(host="0.0.0.0", port=5001)
# lh = LiquidHandler(backend=backend, deck=OTDeck())

The final step is to open communication with the robot. This is done using the {func}`~pylabrobot.liquid_handling.LiquidHandler.setup` method.

In [6]:
await lh.setup()

## Creating the deck layout

Now that we have a `LiquidHandler` instance, we can define the deck layout.

The layout in this tutorial will contain five sets of standard volume tips with filter, 1 set of 96 1mL wells, and tip and plate carriers on which these resources are positioned.

Start by importing the relevant objects and variables from the PyHamilton package. This notebook uses the following resources:

- {class}`~pylabrobot.resources.ml_star.tip_carriers.TIP_CAR_480_A00` tip carrier
- {class}`~pylabrobot.resources.ml_star.plate_carriers.PLT_CAR_L5AC_A00` plate carrier
- {class}`~pylabrobot.resources.corning_costar.plates.Cos_96_DW_1mL` wells
- {class}`~pylabrobot.resources.ml_star.tip_racks.HTF_L` tips

In [7]:
from pylabrobot.resources.opentrons import (
  opentrons_96_filtertiprack_20ul
)
from pylabrobot.resources.corning_costar import (
  Cos_96_EZWash,
  Cos_96_DW_1mL
)


Then create a tip carrier named `tip carrier`, which will contain tip rack at all 5 positions. These positions can be accessed using `tip_car[x]`, and are 0 indexed.

In [8]:
tips1 = opentrons_96_filtertiprack_20ul(name="tip_rack1")
tips2 = opentrons_96_filtertiprack_20ul(name="tip_rack2")
plate1 = Cos_96_EZWash(name="plate_01")
plate2 = Cos_96_DW_1mL(name="plate_02")

Use {func}`~pylabrobot.resources.abstract.assign_child_resources` to assign the tip carrier to the deck of the liquid handler. All resources contained by this carrier will be assigned automatically.

In the `rails` parameter, we can pass the location of the tip carrier. The locations of the tips will automatically be calculated.

In [9]:
lh.deck.assign_child_at_slot(tips1, slot=3)
lh.deck.assign_child_at_slot(tips2, slot=5)
lh.deck.assign_child_at_slot(plate1, slot=11)
lh.deck.assign_child_at_slot(plate2, slot=10)

Repeat this for the plates.

In [10]:
# summary function that displayes deck
print(lh.deck.summary())


Deck: 624.3mm x 565.2mm

+-----------------+-----------------+-----------------+
|                 |                 |                 |
| 10: plate_02    | 11: plate_01    | 12: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  7: Empty       |  8: Empty       |  9: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  4: Empty       |  5: tip_rack2   |  6: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  1: Empty       |  2: Empty       |  3: tip_rack1   |
|                 |                 |                 |
+-----------------+-----------------+-----------------+



In [11]:
# get slot position of resource
lh.deck.get_slot(tips1)

3

Let's look at a summary of the deck layout using {func}`~pylabrobot.liquid_handling.LiquidHandler.summary`.

In [9]:
lh.summary()


Deck: 624.3mm x 565.2mm

+-----------------+-----------------+-----------------+
|                 |                 |                 |
| 10: plate_02    | 11: plate_01    | 12: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  7: Empty       |  8: Empty       |  9: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  4: Empty       |  5: tip_rack2   |  6: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  1: Empty       |  2: Empty       |  3: tip_rack1   |
|                 |                 |                 |
+-----------------+-----------------+-----------------+



## Picking up tips

Picking up tips is as easy as querying the tips from the tiprack.

In [50]:
tiprack = lh.get_resource("tip_rack1")
## OPS OPS CHANGE TO SINGLE CHANNEL OG MULTI CHANNEL FOR THE OT ROBOT
await lh.pick_up_tips(tiprack["A1"])

## Aspirating and dispensing

Aspirating and dispensing work similarly to picking up tips: where you use booleans to specify which tips to pick up, with aspiration and dispensing you use floats to specify the volume to aspirate or dispense in $\mu L$.

The cells below move liquid from wells `'A1:C1'` to `'D1:F1'` using channels 1, 2, and 3 using the {func}`~pylabrobot.liquid_handling.LiquidHandler.aspirate` and {func}`~pylabrobot.liquid_handling.LiquidHandler.dispense` methods.

In [13]:
plate = lh.get_resource("plate_01")
await lh.aspirate(plate["A1"], vols=[15])

After the liquid has been aspirated, dispense it in the wells below. Note that while we specify different wells, we are still using the same channels. This is needed because only these channels contain liquid, of course.

In [14]:
await lh.dispense(plate["F1"], vols=[15])

Let's move the liquid back to the original wells.

In [15]:
# await lh.aspirate(plate["D1:F1"], vols=[100.0, 50.0, 200.0])
# await lh.dispense(plate["A1:C1"], vols=[100.0, 50.0, 200.0])

INFO:pylabrobot.liquid_handling.backends.hamilton.STAR:Sent command: C0ASid0008at0&tm1 1 1 0&xp04330 04330 04330 00000&yp1190 1100 1010 0000&th2450te2450lp1931 1931 1931&ch000 000 000&zl1881 1881 1881&po0100 0100 0100&zu0032 0032 0032&zr06180 06180 06180&zx1831 1831 1831&ip0000 0000 0000&it0 0 0&fp0000 0000 0000&av01072 00551 02110&as1000 1000 1000&ta000 000 000&ba0000 0000 0000&oa000 000 000&lm0 0 0&ll1 1 1&lv1 1 1&zo000 000 000&ld00 00 00&de0020 0020 0020&wt10 10 10&mv00000 00000 00000&mc00 00 00&mp000 000 000&ms1000 1000 1000&mh0000 0000 0000&gi000 000 000&gj0gk0lk0 0 0&ik0000 0000 0000&sd0500 0500 0500&se0500 0500 0500&sz0300 0300 0300&io0000 0000 0000&il00000 00000 00000&in0000 0000 0000&
INFO:pylabrobot.liquid_handling.backends.hamilton.STAR:Received response: C0ASid0008er00/00
INFO:pylabrobot.liquid_handling.backends.hamilton.STAR:Sent command: C0DSid0009dm2 2 2&tm1 1 1 0&xp04330 04330 04330 00000&yp1460 1370 1280 0000&zx1871 1871 1871&lp2321 2321 2321&zl1881 1881 1881&po0100 01

## Discarding tips

Finally, you can discard tips by using the {func}`~pylabrobot.liquid_handling.LiquidHandler.discard_tips` method.

In [15]:
await lh.drop_tips(tiprack["A1"])

# TEST Throughing tip into trash

In [12]:
tiprack = lh.get_resource("tip_rack1")
await lh.pick_up_tips(tiprack["A1"])

In [13]:
from pylabrobot.resources.trash import Trash
# from pylabrobot.resources.opentrons.deck import OTDeck

In [19]:
# trash = lh.deck.get_trash_area()
# # n must match with the number of pipets
# n=1
# OTDeck._assign_trash()
lh.deck._assign_trash
# trash_container.assign_child_resource(actual_trash, location=Coordinate(x=82.84, y=53.56, z=5))
# lh.deck.assign_child_at_slot(lh.deck._assign_trash, 12)


<bound method OTDeck._assign_trash of OTDeck(name=deck, location=(000.000, 000.000, 000.000), size_x=624.3, size_y=565.2, size_z=900, category=deck)>

In [ ]:
# Trash.assign_child_resource(actual_trash, location=Coordinate(x=82.84, y=53.56, z=5))


In [15]:
trash = lh.deck.get_trash_area

In [16]:
print(lh.deck.summary())



Deck: 624.3mm x 565.2mm

+-----------------+-----------------+-----------------+
|                 |                 |                 |
| 10: plate_02    | 11: plate_01    | 12: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  7: Empty       |  8: Empty       |  9: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  4: Empty       |  5: tip_rack2   |  6: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  1: Empty       |  2: Empty       |  3: tip_rack1   |
|                 |                 |                 |
+-----------------+-----------------+-----------------+



In [17]:
await lh.drop_tips(tip_spots=[trash]*1)

TypeError: 'method' object is not iterable

In [19]:
await lh.stop()